In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=50)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
tool.name

'wikipedia'

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://python.langchain.com/docs/how_to/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
db = Chroma.from_documents(documents, OllamaEmbeddings(model="llama3.2"))
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001DC37433010>, search_kwargs={})

In [6]:
from langchain.tools.retriever import create_retriever_tool
retrievel_tool = create_retriever_tool(retriever, "langsmith_search",
                      "Search for information about LangSmith")

In [7]:
retrievel_tool.name

'langsmith_search'

In [8]:
# Arxiv
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [9]:
# Combine all the tools
tools = [tool, arxiv, retrievel_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\asus\\anaconda3\\envs\\llm\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=50)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001DC245A0CC0>, retriever=VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001DC37433010>

In [10]:
import os
from dotenv import load_dotenv
from langchain_ollama import ChatOllama

load_dotenv()
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

llm = ChatOllama(model="llama3.2", temperature=0)

In [12]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [13]:
# Agents
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)

In [14]:
# Agent executer
from langchain.agents import AgentExecutor

agent_executer = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executer

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [16]:
agent_executer.invoke({"input":"Tell me about Deep Learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Deep Learning'}`


Page: Deep learning
Summary: Deep learning is a su**Deep Learning**

Deep learning is a subfield of machine learning that focuses on the use of artificial neural networks (ANNs) with multiple layers to analyze and interpret data. The term "deep" refers to the fact that these ANNs have many layers, typically more than two, which allows them to learn complex patterns in data.

**Key Characteristics:**

1. **Multiple Layers**: Deep learning models consist of multiple layers of interconnected nodes or "neurons," which process and transform inputs into meaningful representations.
2. **Large Amounts of Training Data**: Deep learning models require large amounts of training data to learn effective representations of the input data.
3. **Complexity**: Deep learning models can be computationally expensive to train and require significant computational resources.

**Types of Deep Learning:**

1. **

{'input': 'Tell me about Deep Learning',
 'output': '**Deep Learning**\n\nDeep learning is a subfield of machine learning that focuses on the use of artificial neural networks (ANNs) with multiple layers to analyze and interpret data. The term "deep" refers to the fact that these ANNs have many layers, typically more than two, which allows them to learn complex patterns in data.\n\n**Key Characteristics:**\n\n1. **Multiple Layers**: Deep learning models consist of multiple layers of interconnected nodes or "neurons," which process and transform inputs into meaningful representations.\n2. **Large Amounts of Training Data**: Deep learning models require large amounts of training data to learn effective representations of the input data.\n3. **Complexity**: Deep learning models can be computationally expensive to train and require significant computational resources.\n\n**Types of Deep Learning:**\n\n1. **Convolutional Neural Networks (CNNs)**: Designed for image and video processing, CNN